In [1]:
import numpy as np
import pandas as pd

In [2]:
from lightautoml_gpu.reader.gpu.cudf_reader import CudfReader
from lightautoml_gpu.reader.base import PandasToPandasReader

from lightautoml_gpu.transformers.base import SequentialTransformer

from lightautoml_gpu.pipelines.utils import get_columns_by_role

from lightautoml_gpu.transformers.gpu import numeric_gpu, categorical_gpu, datetime_gpu
from lightautoml_gpu.transformers import numeric, categorical, datetime

from lightautoml_gpu.tasks import Task
from lightautoml_gpu.automl.presets.tabular_presets import TabularAutoML, TabularUtilizedAutoML

import pandas as pd
import time
from sklearn.model_selection import train_test_split

from dask.distributed import Client
from dask_cuda import LocalCUDACluster
import cudf

In [3]:
features = pd.read_csv('../../data/small_new/multilabel/train Data.csv')
labels = pd.read_csv('../../data/small_new/multilabel/train labels.csv')
labels.drop(columns='id', inplace=True)
data = pd.concat([features, labels], axis=1)
data = data[['n_0000','n_0001','n_0002','n_0003',
             'n_0004','n_0005','n_0006','n_0007',
             'service_a', 'service_b', 'service_c', 
             'service_d', 'service_e', 'service_f',
             'service_g', 'service_h', 'service_i',
             'service_j', 'service_k', 'service_l',
             'service_m', 'service_n']]

tr_data, te_data = train_test_split(
    data, 
    test_size=0.2,  
    random_state=42
)

roles = {
    "target": {'service_a', 'service_b', 'service_c',
               'service_d', 'service_e', 'service_f',
               'service_g', 'service_h', 'service_i',
               'service_j', 'service_k', 'service_l',
               'service_m', 'service_n'},
    "drop" : ['id']
}

/tmp/ipykernel_10147/705777730.py:1: DtypeWarning: Columns (329,331,333,336,338,344,345,346,348,354,355,356,357,358,361,362,364,367,372,377,380,383,385,387,390,392,399,400,406,408,409,413,416,418,419,431,433,437,438,442,448,449,450,453,457,464,473,478,479,481,483,485,486,489,492,495,496,497,498,499,500,503,507,508,510,511,514,515,517,519,520,521,522,523,524,526,527,530,534,537,538,539,541,544,547,548,549,550,551,557,558,560,564,569,572,573,576,577,579,583,596,597,598,600,601,602,604,605,606,608,609,613,619,620,625,627,628,629,631,632,633,636,641,642,643,645,646,647,648,651,655,661,662,665,668,675,676,679,682,685,687,689,690,691,694,698,701,702,703,706,711,712,713,719,720,721,733,735,737,738,742,746,747,748,749,752,754,755,760,764,768,770,781,782,789,797,807,812,814,817,818,822,823,824,825,832,840,843,844,845,850,853,857,858,861,867,868,873,874,876,877,879,880,881,883,886,890,893,897,899,900,901,902,904,905,908,909,910,912,913,914,915,916,922,923,931,933,935,937,939,942,943,946,951,955,

In [4]:
adv_roles = True

## Imports (for potential use)

In [5]:
# Imports from our package
from lightautoml_gpu.automl.base import AutoML

from lightautoml_gpu.automl.presets.gpu.tabular_gpu_presets import TabularAutoMLGPU, TabularUtilizedAutoMLGPU
from lightautoml_gpu.tasks import Task

from lightautoml_gpu.pipelines.features.gpu.lgb_pipeline_gpu import LGBSimpleFeaturesGPU, LGBAdvancedPipelineGPU
from lightautoml_gpu.pipelines.features.gpu.linear_pipeline_gpu import LinearFeaturesGPU

from lightautoml_gpu.pipelines.features.lgb_pipeline import LGBSimpleFeatures, LGBAdvancedPipeline
from lightautoml_gpu.pipelines.features.linear_pipeline import LinearFeatures


from lightautoml_gpu.ml_algo.gpu.boost_cb_gpu import BoostCBGPU
from lightautoml_gpu.ml_algo.gpu.boost_xgb_gpu import BoostXGB
from lightautoml_gpu.ml_algo.gpu.linear_gpu import LinearLBFGSGPU

from lightautoml_gpu.ml_algo.boost_cb import BoostCB
from lightautoml_gpu.ml_algo.linear_sklearn import LinearLBFGS


from lightautoml_gpu.pipelines.ml.base import MLPipeline
from lightautoml_gpu.pipelines.selection.importance_based import ModelBasedImportanceEstimator, ImportanceCutoffSelector

## TabularAutoML

In [6]:
task = Task('multilabel')

multilabel isn`t supported in lgb


In [7]:
automl = TabularAutoML(
    task = task, 
    timeout = 3600,
    cpu_limit = 4,
    reader_params = {'n_jobs': 4, 'cv': 3, 'random_state': 42},
    general_params = {'use_algos': [['linear_l2', 'cb']]}
)

In [8]:
oof_pred = automl.fit_predict(data, roles = roles, verbose = 2)

[12:13:22] Stdout logging level is INFO2.
[12:13:22] Copying TaskTimer may affect the parent PipelineTimer, so copy will create new unlimited TaskTimer
[12:13:22] Task: multilabel

[12:13:22] Start automl preset with listed constraints:
[12:13:22] - time: 3600.00 seconds
[12:13:22] - CPU: 4 cores
[12:13:22] - memory: 16 GB

[12:13:22] Train data shape: (14644, 22)



/home/rishat/miniconda3/envs/rapids-22.10/lib/python3.9/site-packages/lightautoml_gpu/reader/guess_roles.py:56: RuntimeWarning: invalid value encountered in float_scalars
  gini_sum = a_c.sum() / a_s.sum() - (n + 1) / 2.0
/home/rishat/miniconda3/envs/rapids-22.10/lib/python3.9/site-packages/lightautoml_gpu/reader/guess_roles.py:56: RuntimeWarning: invalid value encountered in float_scalars
  gini_sum = a_c.sum() / a_s.sum() - (n + 1) / 2.0
/home/rishat/miniconda3/envs/rapids-22.10/lib/python3.9/site-packages/lightautoml_gpu/reader/guess_roles.py:56: RuntimeWarning: invalid value encountered in float_scalars
  gini_sum = a_c.sum() / a_s.sum() - (n + 1) / 2.0
/home/rishat/miniconda3/envs/rapids-22.10/lib/python3.9/site-packages/lightautoml_gpu/reader/guess_roles.py:56: RuntimeWarning: invalid value encountered in float_scalars
  gini_sum = a_c.sum() / a_s.sum() - (n + 1) / 2.0


[12:13:26] Feats was rejected during automatic roles guess: []
[12:13:26] Layer 1 train process start. Time left 3595.87 secs


/home/rishat/miniconda3/envs/rapids-22.10/lib/python3.9/site-packages/lightautoml_gpu/reader/guess_roles.py:56: RuntimeWarning: invalid value encountered in float_scalars
  gini_sum = a_c.sum() / a_s.sum() - (n + 1) / 2.0
/home/rishat/miniconda3/envs/rapids-22.10/lib/python3.9/site-packages/lightautoml_gpu/reader/guess_roles.py:56: RuntimeWarning: invalid value encountered in float_scalars
  gini_sum = a_c.sum() / a_s.sum() - (n + 1) / 2.0
/home/rishat/miniconda3/envs/rapids-22.10/lib/python3.9/site-packages/lightautoml_gpu/reader/guess_roles.py:56: RuntimeWarning: invalid value encountered in float_scalars
  gini_sum = a_c.sum() / a_s.sum() - (n + 1) / 2.0
/home/rishat/miniconda3/envs/rapids-22.10/lib/python3.9/site-packages/lightautoml_gpu/reader/guess_roles.py:56: RuntimeWarning: invalid value encountered in float_scalars
  gini_sum = a_c.sum() / a_s.sum() - (n + 1) / 2.0


[12:13:26] Start fitting Lvl_0_Pipe_0_Mod_0_LinearL2 ...
[12:13:26] ===== Start working with fold 0 for Lvl_0_Pipe_0_Mod_0_LinearL2 =====
[12:13:29] ===== Start working with fold 1 for Lvl_0_Pipe_0_Mod_0_LinearL2 =====
[12:13:31] ===== Start working with fold 2 for Lvl_0_Pipe_0_Mod_0_LinearL2 =====
[12:13:34] Fitting Lvl_0_Pipe_0_Mod_0_LinearL2 finished. score = -7.349995876634196
[12:13:34] Lvl_0_Pipe_0_Mod_0_LinearL2 fitting and predicting completed
[12:13:34] Time left 3588.46 secs



...

[12:13:52] ===== Start working with fold 1 for Lvl_0_Pipe_1_Mod_0_CatBoost =====


/home/rishat/miniconda3/envs/rapids-22.10/lib/python3.9/site-packages/catboost/core.py:1419: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,
/home/rishat/miniconda3/envs/rapids-22.10/lib/python3.9/site-packages/catboost/core.py:1419: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,


[12:14:01] ===== Start working with fold 2 for Lvl_0_Pipe_1_Mod_0_CatBoost =====


/home/rishat/miniconda3/envs/rapids-22.10/lib/python3.9/site-packages/catboost/core.py:1419: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,
/home/rishat/miniconda3/envs/rapids-22.10/lib/python3.9/site-packages/catboost/core.py:1419: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,


[12:14:11] Fitting Lvl_0_Pipe_1_Mod_0_CatBoost finished. score = -7.322678701041892
[12:14:11] Lvl_0_Pipe_1_Mod_0_CatBoost fitting and predicting completed
[12:14:11] Time left 3551.41 secs

[12:14:11] Layer 1 training completed.

[12:14:11] Blending: optimization starts with equal weights and score -7.316072528962554
[12:14:11] Blending: iteration 0: score = -7.31587541370778, weights = [0.5771486 0.4228514]
[12:14:11] Blending: iteration 1: score = -7.31587541370778, weights = [0.5771486 0.4228514]
[12:14:11] Blending: no score update. Terminated

[12:14:11] Automl preset training completed in 49.00 seconds

[12:14:11] Model description:
Final prediction for new objects (level 0) = 
	 0.57715 * (3 averaged models Lvl_0_Pipe_0_Mod_0_LinearL2) +
	 0.42285 * (3 averaged models Lvl_0_Pipe_1_Mod_0_CatBoost) 



In [9]:
task = Task('multilabel', device='gpu')

multilabel isn`t supported in lgb
[12:14:13] CatBoost uses as obj. MultiCrossEntropy.


In [10]:
automl_gpu = TabularAutoMLGPU(
    task = task, 
    timeout = 3600,
    cpu_limit = 1,
    reader_params = {'n_jobs': 1, 'cv': 3, 'random_state': 42},
    general_params = {'use_algos': [['xgb', 'linear_l2']]}
)

In [11]:
oof_pred_gpu = automl_gpu.fit_predict(data, roles = roles, verbose = 2)

[12:14:13] Stdout logging level is INFO2.
[12:14:13] Task: multilabel

[12:14:13] Start automl preset with listed constraints:
[12:14:13] - time: 3600.00 seconds
[12:14:13] - CPU: 1 cores
[12:14:13] - memory: 16 GB

[12:14:13] Train data shape: (14644, 22)
[12:14:14] Feats was rejected during automatic roles guess: []
[12:14:14] Layer 1 train process start. Time left 3599.29 secs
[12:14:14] Start fitting Lvl_0_Pipe_0_Mod_0_LinearL2 ...
[12:14:14] ===== Start working with fold 0 for Lvl_0_Pipe_0_Mod_0_LinearL2 (orig) =====
[12:14:21] ===== Start working with fold 1 for Lvl_0_Pipe_0_Mod_0_LinearL2 (orig) =====
[12:14:25] ===== Start working with fold 2 for Lvl_0_Pipe_0_Mod_0_LinearL2 (orig) =====
[12:14:32] Fitting Lvl_0_Pipe_0_Mod_0_LinearL2 finished. score = -7.37759298422165
[12:14:32] Lvl_0_Pipe_0_Mod_0_LinearL2 fitting and predicting completed
[12:14:32] Time left 3581.21 secs

[12:14:34] Stdout logging level is INFO.
[12:14:34] GDBT train starts. Max iter 3000, early stopping round

In [12]:
gpu_inf = automl_gpu.predict(data)

In [13]:
automl_gpu.to_cpu()

multilabel isn`t supported in lgb
[12:16:13] CatBoost uses as obj. MultiCrossEntropy.
multilabel isn`t supported in lgb
[12:16:14] CatBoost uses as obj. MultiCrossEntropy.


In [14]:
cpu_inf = automl_gpu.predict(data)

In [27]:
from joblib import dump, load
import time
pickle_file = './gpu.joblib'

In [23]:
start = time.time()
with open(pickle_file, 'wb') as f:
    dump(automl_mgpu, f)
raw_dump_duration = time.time() - start
print("Raw dump duration: %0.3fs" % raw_dump_duration)

Raw dump duration: 0.434s


In [28]:

start = time.time()
with open(pickle_file, 'rb') as f:
    model_inf = load(f)
raw_load_duration = time.time() - start
print("Raw load duration: %0.3fs" % raw_load_duration)


Raw load duration: 0.321s


In [29]:
disk_pred = model_inf.predict(data)

In [26]:
disk_pred.data.T

array([[0.02250414, 0.03473555, 0.02189182, ..., 0.02092039, 0.02245283,
        0.02958762],
       [0.10242204, 0.10307302, 0.08328716, ..., 0.18352863, 0.05845414,
        0.08106075],
       [0.01661056, 0.02685064, 0.01843996, ..., 0.01261119, 0.01576334,
        0.02525279],
       ...,
       [0.06970395, 0.10045627, 0.07541621, ..., 0.12828714, 0.06813928,
        0.06517109],
       [0.00863461, 0.0093553 , 0.00729773, ..., 0.00250403, 0.01063326,
        0.00911328],
       [0.37738806, 0.32151622, 0.36710003, ..., 0.19072054, 0.38499045,
        0.348848  ]], dtype=float32)

In [30]:
disk_pred.data.T

array([[0.02228719, 0.03408891, 0.0235206 , ..., 0.02092   , 0.02458438,
        0.03057472],
       [0.10153766, 0.10434376, 0.08275151, ..., 0.18232661, 0.05773228,
        0.08430795],
       [0.0170659 , 0.02848539, 0.01784247, ..., 0.01342579, 0.0159637 ,
        0.02606961],
       ...,
       [0.06861058, 0.09899426, 0.07224359, ..., 0.12805845, 0.06901285,
        0.06687039],
       [0.00891042, 0.01045184, 0.00740691, ..., 0.0024012 , 0.01186853,
        0.00778994],
       [0.37658358, 0.31884933, 0.36671746, ..., 0.1914039 , 0.38522282,
        0.347609  ]], dtype=float32)

In [ ]:
print(cpu_inf.data.T)
print()
print(gpu_inf.data.T)
print()
print(oof_pred_gpu.data.T)
print()
print(oof_pred.data.T)
print()
print(disk_pred.data.T)

In [15]:
cluster = LocalCUDACluster(rmm_managed_memory=True, CUDA_VISIBLE_DEVICES="0",
                               protocol="ucx", enable_nvlink=True,
                               memory_limit="8GB")
print("dashboard:", cluster.dashboard_link)
client = Client(cluster)
client.run(cudf.set_allocator, "managed")

/home/rishat/miniconda3/envs/rapids-22.10/lib/python3.9/site-packages/distributed/comm/ucx.py:61: UserWarning: A CUDA context for device 0 already exists on process ID 10147. This is often the result of a CUDA-enabled library calling a CUDA runtime function before Dask-CUDA can spawn worker processes. Please make sure any such function calls don't happen at import time or in the global scope of a program.
  warnings.warn(
2022-12-27 12:16:46,339 - distributed.preloading - INFO - Creating preload: dask_cuda.initialize
2022-12-27 12:16:46,339 - distributed.preloading - INFO - Import preload module: dask_cuda.initialize


dashboard: http://127.0.0.1:8787/status


{'ucx://127.0.0.1:45803': None}

In [16]:
task = Task('multilabel', device='mgpu')

automl_mgpu = TabularAutoMLGPU(
    task = task, 
    timeout = 3600,
    cpu_limit = 1,
    reader_params = {'n_jobs': 1, 'cv': 3, 'random_state': 42, 'npartitions': 2},
    general_params = {'use_algos': [['xgb', 'linear_l2']]},
    client = client
)

oof_pred_mgpu = automl_mgpu.fit_predict(data, roles = roles, verbose = 2)

multilabel isn`t supported in lgb
[12:16:47] CatBoost uses as obj. MultiCrossEntropy.
[12:16:47] Stdout logging level is INFO2.
[12:16:47] Task: multilabel

[12:16:47] Start automl preset with listed constraints:
[12:16:47] - time: 3600.00 seconds
[12:16:47] - CPU: 1 cores
[12:16:47] - memory: 16 GB

[12:16:47] Train data shape: (14644, 22)
[12:16:49] Feats was rejected during automatic roles guess: []
[12:16:49] Layer 1 train process start. Time left 3598.48 secs
[12:16:50] Start fitting Lvl_0_Pipe_0_Mod_0_LinearL2 ...
[12:16:50] ===== Start working with fold 0 for Lvl_0_Pipe_0_Mod_0_LinearL2 (orig) =====
[12:16:57] ===== Start working with fold 1 for Lvl_0_Pipe_0_Mod_0_LinearL2 (orig) =====
[12:17:03] ===== Start working with fold 2 for Lvl_0_Pipe_0_Mod_0_LinearL2 (orig) =====
[12:17:14] Fitting Lvl_0_Pipe_0_Mod_0_LinearL2 finished. score = -7.376927413238679
[12:17:14] Lvl_0_Pipe_0_Mod_0_LinearL2 fitting and predicting completed
[12:17:14] Time left 3573.72 secs

[12:17:14] Stdout l

/home/rishat/miniconda3/envs/rapids-22.10/lib/python3.9/site-packages/xgboost/dask.py:884: RuntimeWarning: coroutine 'Client._wait_for_workers' was never awaited
  client.wait_for_workers(n_workers)
/home/rishat/miniconda3/envs/rapids-22.10/lib/python3.9/site-packages/distributed/worker_state_machine.py:3649: FutureWarning: The `Worker.nthreads` attribute has been moved to `Worker.state.nthreads`
  warnings.warn(
[12:17:32] task [xgboost.dask-0]:ucx://127.0.0.1:45803 got new rank 0


[12:18:11] ===== Start working with fold 1 for Lvl_0_Pipe_1_Mod_0_XGB (orig) =====


/home/rishat/miniconda3/envs/rapids-22.10/lib/python3.9/site-packages/xgboost/dask.py:884: RuntimeWarning: coroutine 'Client._wait_for_workers' was never awaited
  client.wait_for_workers(n_workers)
/home/rishat/miniconda3/envs/rapids-22.10/lib/python3.9/site-packages/distributed/worker_state_machine.py:3649: FutureWarning: The `Worker.nthreads` attribute has been moved to `Worker.state.nthreads`
  warnings.warn(
[12:18:11] task [xgboost.dask-0]:ucx://127.0.0.1:45803 got new rank 0


[12:18:48] ===== Start working with fold 2 for Lvl_0_Pipe_1_Mod_0_XGB (orig) =====


/home/rishat/miniconda3/envs/rapids-22.10/lib/python3.9/site-packages/xgboost/dask.py:884: RuntimeWarning: coroutine 'Client._wait_for_workers' was never awaited
  client.wait_for_workers(n_workers)
/home/rishat/miniconda3/envs/rapids-22.10/lib/python3.9/site-packages/distributed/worker_state_machine.py:3649: FutureWarning: The `Worker.nthreads` attribute has been moved to `Worker.state.nthreads`
  warnings.warn(
[12:18:49] task [xgboost.dask-0]:ucx://127.0.0.1:45803 got new rank 0


[12:19:20] Fitting Lvl_0_Pipe_1_Mod_0_XGB finished. score = -7.421265012573688
[12:19:20] Lvl_0_Pipe_1_Mod_0_XGB fitting and predicting completed
[12:19:20] Time left 3447.15 secs

[12:19:20] Layer 1 training completed.

[12:19:20] Blending: Optimization starts with equal weights and score -7.372191601671732
[12:19:20] Blending, iter 0: score = -7.333974977133542, weights = [0.90983003 0.09016994]
[12:19:20] Blending, iter 1: score = -7.333974977133542, weights = [0.90983003 0.09016994]
[12:19:20] No score update. Terminated
[12:19:20] Automl preset training completed in 153.01 seconds

[12:19:20] Model description:
Final prediction for new objects (level 0) = 
	 0.90983 * (3 averaged models Lvl_0_Pipe_0_Mod_0_LinearL2) +
	 0.09017 * (3 averaged models Lvl_0_Pipe_1_Mod_0_XGB) 



In [17]:
automl_mgpu.to_cpu()

multilabel isn`t supported in lgb
[12:19:20] CatBoost uses as obj. MultiCrossEntropy.
multilabel isn`t supported in lgb
[12:19:21] CatBoost uses as obj. MultiCrossEntropy.


In [18]:
mcpu_inf = automl_mgpu.predict(data)

In [19]:
print(cpu_inf.data.T)
print()
print(gpu_inf.data.T)
print()
print(oof_pred_gpu.data.T)
print()
print(oof_pred.data.T)
print()
print(oof_pred_mgpu.data.T)
print()
print(mcpu_inf.data.T)

[[0.02228719 0.03408891 0.0235206  ... 0.02092    0.02458438 0.03057472]
 [0.10153766 0.10434376 0.08275151 ... 0.18232661 0.05773228 0.08430795]
 [0.0170659  0.02848539 0.01784247 ... 0.01342579 0.0159637  0.02606961]
 ...
 [0.06861058 0.09899426 0.07224359 ... 0.12805845 0.06901285 0.06687039]
 [0.00891042 0.01045184 0.00740691 ... 0.0024012  0.01186853 0.00778994]
 [0.37658358 0.31884933 0.36671746 ... 0.1914039  0.38522282 0.347609  ]]

[[0.02150002 0.03343973 0.02268486 ... 0.02115737 0.02309219 0.02931143]
 [0.08235467 0.08712235 0.07123534 ... 0.17138526 0.05641583 0.08124839]
 [0.01663294 0.02748356 0.017249   ... 0.0121594  0.01553418 0.02698088]
 ...
 [0.06330457 0.09227008 0.06795748 ... 0.15138614 0.06560011 0.07477207]
 [0.00655559 0.00727603 0.00565246 ... 0.00238839 0.00824329 0.00563865]
 [0.37649012 0.3263177  0.37465692 ... 0.19588684 0.39084518 0.3470318 ]]

[[0.02311882 0.0344336  0.02351186 ... 0.02095252 0.02601094 0.03094913]
 [0.07560191 0.07959972 0.0788729  ..

In [ ]:
from joblib import dump, load
import time

pickle_file = './gpu.joblib'

start = time.time()
with open(pickle_file, 'wb') as f:
    dump(automl_mgpu.client, f)
raw_dump_duration = time.time() - start
print("Raw dump duration: %0.3fs" % raw_dump_duration)

In [ ]:
automl_mgpu.levels[0][0].ml_algos